In [199]:

import numpy as np
import pandas as pd
from numpy import log2 as log
eps = np.finfo(float).eps
import random
import math

data = pd.read_csv('./dataset/train.csv')
data['Age'] = data['Age'].interpolate()
data.dropna()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [200]:
data.loc[data['Age'] <= 16, 'Age'] = 0
data.loc[(data['Age'] > 16) & (data['Age'] < 50), 'Age'] = 1
data.loc[data['Age'] >= 50, 'Age'] = 2
data.loc[data['Fare'] <=15.64585, 'Fare'] = 0
data.loc[(data['Fare'] > 15.64585) & (data['Fare'] < 33), 'Fare'] = 1
data.loc[data['Fare'] >= 33, 'Fare'] = 2
data['Embarked'] = data['Embarked'].fillna(method = 'ffill')
data = data.drop(['Name', 'Ticket', 'Cabin', 'PassengerId', 'SibSp', 'Parch'], axis = 1)

In [201]:
def entropy(probs):  
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

In [210]:
def attribute_entropy(col):
    p = col.value_counts(normalize = True)
    return entropy(p)
print(attribute_entropy(data['Sex']))

0.9362046432498521


In [211]:
def split_and_findgain(d, node):
    
    subtable = d.groupby(node)
    all_entropies = subtable.agg(attribute_entropy)
    #print((all_entropies))
    vals = d[node].value_counts(normalize = True)
    
    wt_avg = sum(all_entropies['Survived'] * vals)
    #print("Average entropy :", wt_avg)
    entropy = attribute_entropy(d['Survived'])
    #print("Entropy :", entropy)
    ig = entropy - wt_avg
    #print(ig)
    return ig
print(split_and_findgain(data,'Sex'))

0.2176601066606142


In [213]:
def buildTree(d, attributes, return_val = None):
    
    proportion = {'0' : list(d['Survived']).count(0), '1' : list(d['Survived']).count(1)}
    
    if len(proportion) == 1:
        return proportion.keys()
    
    elif d.empty or(not attributes):
        return return_val

    else:
        return_val = max(proportion.keys())
        ig = []
        for node in attributes:
            ig.append(split_and_findgain(d, node))
        print(ig)
        node = attributes[ig.index(max(ig))]
        print(node)
        tree = {node :{}}

        att = attributes.copy()
        att.remove(node)
        print(att)
        
        for val, subtable in d.groupby(node):
            print(tree)
            tree[node][val] = buildTree(subtable, att, return_val)
        return tree

print(buildTree(data, ['Pclass', 'Age', 'Fare', 'Sex', 'Embarked']))

[0.0838310452960116, 0.009113060149028573, 0.06632324196798867, 0.2176601066606142, 0.021583390635887656]
Sex
['Pclass', 'Age', 'Fare', 'Embarked']
{'Sex': {}}
[0.207572089443242, 0.013371856074800403, 0.04850863553729379, 0.025802330202799206]
Pclass
['Age', 'Fare', 'Embarked']
{'Pclass': {}}
[0.018090126658824146, 0.011856716605231354, 0.003055270995843795]
Age
['Fare', 'Embarked']
{'Age': {}}
[0.0, 0.10218717094933333]
Embarked
['Fare']
{'Embarked': {}}
[0.0]
Fare
[]
{'Fare': {}}
{'Embarked': {'C': {'Fare': {2.0: '1'}}}}
[0.0]
Fare
[]
{'Fare': {}}
{'Age': {0.0: {'Embarked': {'C': {'Fare': {2.0: '1'}}, 'S': {'Fare': {2.0: '1'}}}}}}
[0.0019166721182913293, 0.013743765997408147]
Embarked
['Fare']
{'Embarked': {}}
[0.0]
Fare
[]
{'Fare': {}}
{'Fare': {1.0: '1'}}
{'Embarked': {'C': {'Fare': {1.0: '1', 2.0: '1'}}}}
[0.0]
Fare
[]
{'Fare': {}}
{'Embarked': {'C': {'Fare': {1.0: '1', 2.0: '1'}}, 'Q': {'Fare': {2.0: '1'}}}}
[0.004791113356737514]
Fare
[]
{'Fare': {}}
{'Fare': {1.0: '1'}}
{'Age'

[0.1476625692853939, 0.027740167559621387]
Fare
['Embarked']
{'Fare': {}}
[0.0]
Embarked
[]
{'Embarked': {}}
{'Fare': {0.0: {'Embarked': {'S': '1'}}}}
[0.0]
Embarked
[]
{'Embarked': {}}
{'Fare': {0.0: {'Embarked': {'S': '1'}}, 1.0: {'Embarked': {'S': '1'}}}}
[0.0]
Embarked
[]
{'Embarked': {}}
{'Embarked': {'C': '1'}}
{'Age': {0.0: {'Fare': {0.0: {'Embarked': {'S': '1'}}, 1.0: {'Embarked': {'S': '1'}}, 2.0: {'Embarked': {'C': '1', 'S': '1'}}}}}}
[0.020100588158356747, 0.0008077517542137702]
Fare
['Embarked']
{'Fare': {}}
[0.009071134410195358]
Embarked
[]
{'Embarked': {}}
{'Embarked': {'C': '1'}}
{'Fare': {0.0: {'Embarked': {'C': '1', 'S': '1'}}}}
[0.01205354205316772]
Embarked
[]
{'Embarked': {}}
{'Embarked': {'C': '1'}}
{'Fare': {0.0: {'Embarked': {'C': '1', 'S': '1'}}, 1.0: {'Embarked': {'C': '1', 'S': '1'}}}}
[0.0]
Embarked
[]
{'Embarked': {}}
{'Embarked': {'C': '1'}}
{'Age': {0.0: {'Fare': {0.0: {'Embarked': {'S': '1'}}, 1.0: {'Embarked': {'S': '1'}}, 2.0: {'Embarked': {'C': '1', '